<a href="https://colab.research.google.com/github/smartkorea/nlp/blob/main/nlp_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
cd /content/gdrive/MyDrive/eng_sa

/content/gdrive/MyDrive/eng_sa


In [3]:
!pip install transformers==2.9.1

     |████████████████████████████████| 645kB 15.7MB/s 
     |████████████████████████████████| 3.8MB 33.2MB/s 
     |████████████████████████████████| 890kB 77.6MB/s 
     |████████████████████████████████| 1.1MB 59.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4399c5bc226dda75680102e57f529199850d7871a62279981f335f2ae295a79c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [17]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random
import time
import datetime
import json
import re
import nltk
import matplotlib.pyplot as plt
import os
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))
stemmer = nltk.stem.SnowballStemmer('english')
with open('./friends_train.json', encoding='utf-8') as json_file:
    json_train = json.load(json_file)


def cleaning(str):
    replaceAll= str
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)
    no_capitals = only_english.lower().split()
    no_stops = [word for word in no_capitals if not word in stops]
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    #return ' '.join(stemmer_words)
    return ' '.join(no_capitals)

i = 0
train_data=[]
for rows in json_train:
    for row in rows:
        #train_data.append([cleaning(row['utterance']), row['emotion']])
        train_data.append([cleaning(row['utterance']), row['emotion']])
    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
train_data[10:30]

[['no don t i beg of you', 'fear'],
 ['all right then we ll have a definite answer for you on monday but i think i can say with some confidence you ll fit in well here',
  'neutral'],
 ['really', 'surprise'],
 ['absolutely you can relax you did great', 'neutral'],
 ['but then who the waitress i went out with last month', 'surprise'],
 ['you know forget it', 'non-neutral'],
 ['no no no no no who who were you talking about', 'surprise'],
 ['no i i i i don t i actually don t know', 'non-neutral'],
 ['ok', 'neutral'],
 ['all right well', 'neutral'],
 ['i m gonna see if i can get a room for the night and i ll', 'non-neutral'],
 ['i ll see you later', 'non-neutral'],
 ['yeah sure', 'neutral'],
 ['hey mon', 'neutral'],
 ['hey hey hey you wanna hear something that sucks', 'neutral'],
 ['do i ever', 'joy'],
 ['chris says they re closing down the bar', 'sadness'],
 ['no way', 'surprise'],
 ['yeah apparently they re turning it into some kinda coffee place',
  'neutral'],
 ['just coffee where are 

In [20]:
sentences = []
for i in train_data:
  sentences.append(i[0])
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

In [21]:
maxlen = 0
for i in sentences:
  if maxlen < len(i):
    maxlen = len(i)

MAX_LEN = maxlen + 1
#MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)


In [22]:
def labeltoint(str):
    return {'non-neutral': 0,
             'neutral': 1, 
             'joy': 2,
             'sadness': 3,
             'fear': 4,
             'anger': 5,
             'surprise': 6,
             'disgust': 7}[str]

labels = []
for i in train_data:
  labels.append(labeltoint(i[1]))

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=41, 
                                                                                    test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=41, 
                                                       test_size=0.2)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)		

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [10]:
train_data[10:20]

(tensor([[  101, 20878, 10887,  ...,     0,     0,     0],
         [  101, 26114, 26114,  ...,     0,     0,     0],
         [  101,  4826,  2012,  ...,     0,     0,     0],
         ...,
         [  101,  1045,  2147,  ...,     0,     0,     0],
         [  101,  1037,  2733,  ...,     0,     0,     0],
         [  101,  1045,  2123,  ...,     0,     0,     0]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 tensor([0, 0, 1, 1, 2, 2, 2, 1, 0, 5]))

In [ ]:
 os.path.isdir("./bert_model")

True

In [23]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=8)
#model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=8)
model.cuda()

optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                  ,correct_bias=False
                )
epochs = 5
total_steps = len(train_dataloader) * epochs
# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))



Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [24]:
# 재현을 위해 랜덤시드 고정
seed_val = 41
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))



# model 저장
model.save_pretrained('./bert_model/')


print("")
print("Training complete!")


======== Epoch 1 / 5 ========
Training...


/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)



  Average training loss: 1.54
  Training epcoh took: 0:04:29

Running Validation...
  Accuracy: 0.54
  Validation took: 0:00:22

======== Epoch 2 / 5 ========
Training...

  Average training loss: 1.28
  Training epcoh took: 0:04:28

Running Validation...
  Accuracy: 0.54
  Validation took: 0:00:22

======== Epoch 3 / 5 ========
Training...

  Average training loss: 1.01
  Training epcoh took: 0:04:29

Running Validation...
  Accuracy: 0.55
  Validation took: 0:00:22

======== Epoch 4 / 5 ========
Training...

  Average training loss: 0.76
  Training epcoh took: 0:04:28

Running Validation...
  Accuracy: 0.53
  Validation took: 0:00:22

======== Epoch 5 / 5 ========
Training...

  Average training loss: 0.61
  Training epcoh took: 0:04:28

Running Validation...
  Accuracy: 0.52
  Validation took: 0:00:22

Training complete!


In [ ]:
model = BertForSequenceClassification.from_pretrained('./bert_model/', num_labels=8)

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [25]:
device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


with open('en_data.csv', 'r', encoding='utf-8', newline='') as csvfile:
  en = pd.read_csv(csvfile)

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [26]:
en[10:20]

,id,i_dialog,i_utterance,speaker,utterance
10,10,1,5,Ross,You were trying to eat it!
11,11,1,6,Chandler,If
12,12,1,7,Monica,Come on. Hello? I?m sorry you have the wrong...
13,13,2,0,Julie,"I was thinking of doing it a little shorter, y..."
14,14,2,1,Phoebe,"Oh yeah!? Oh, I can do that."
15,15,2,2,Julie,Really?
16,16,2,3,Hombre Man,"I'm sorry. I am such a doofus. I'm so sorry, I..."
17,17,2,4,Annabelle,"My god, what happened?"
18,18,2,5,Joey,"These new kids, they never last."
19,19,2,6,Joey,"Sooner or later, they all...stop lastin'."


In [ ]:
en['utterance']

0                           Alright, whadyou do with him?
1                                       Oh! You're awake!
2       Then you gotta come clean with Ma! This is not...
3                                       Yeah, but this is
4               I don't wanna hear it! Now go to my room!
                              ...                        
1618                                                Nooo.
1619                                            Hi, Kate!
1620                                          Hi, Lauren.
1621                                          Hi, Lauren.
1622                                             Hi, pig!
Name: utterance, Length: 1623, dtype: object

In [27]:
i = 0
test_data=[]
for ind in en.index:
  #print(cleaning(en['utterance'][ind]))
  test_data.append([cleaning(en['utterance'][ind])])

In [28]:
test_data[10:20]

[['you were trying to eat it'],
 ['if'],
 ['come on hello i m sorry you have the wrong number okay i ll call you later dad i love you'],
 ['i was thinking of doing it a little shorter you know like andie mcdowell s new haircut'],
 ['oh yeah oh i can do that'],
 ['really'],
 ['i m sorry i am such a doofus i m so sorry i m so sorry'],
 ['my god what happened'],
 ['these new kids they never last'],
 ['sooner or later they all stop lastin']]

['you were trying to eat it']

In [ ]:
def cleaning(str):
    replaceAll= str
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)
    no_capitals = only_english.lower().split()
    no_stops = [word for word in no_capitals if not word in stops]
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    #return ' '.join(stemmer_words)
    return ' '.join(no_capitals)

i = 0
test_data=[]
for rows in en:
    #for row in rows:
        #train_data.append([cleaning(row['utterance']), row['emotion']])
        #test_data.append([cleaning(row['utterance']), row['emotion']])

id
i_dialog
i_utterance
speaker
utterance


In [29]:
model.cuda()

def test_sentences(sentences):
    model.eval()
    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    return logits

def convert_input_data(sentences):
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

def inttolabel(idx):
    return {0:'non-neutral',
             1:'neutral', 
             2:'joy',
             3:'sadness',
             4:'fear',
             5:'anger',
             6:'surprise',
             7:'disgust'}[idx]

en_data = [['Id', 'Predicted']]

for idx in range(len(test_data)):
  sen = test_data[idx]
  #sen = cleaning(sen)
  #logit = test_sentences([sen])
  logit = test_sentences(sen)

  en_data.append([idx, inttolabel(np.argmax(logit))])

dataframe = pd.DataFrame(en_data)
dataframe.to_csv("sample.csv", header=False, index=False, mode='w+')

In [ ]:
en_data[10:30]

[[9, 'neutral'],
 [10, 'non-neutral'],
 [11, 'neutral'],
 [12, 'sadness'],
 [13, 'neutral'],
 [14, 'neutral'],
 [15, 'neutral'],
 [16, 'neutral'],
 [17, 'non-neutral'],
 [18, 'neutral'],
 [19, 'neutral'],
 [20, 'neutral'],
 [21, 'non-neutral'],
 [22, 'surprise'],
 [23, 'non-neutral'],
 [24, 'joy'],
 [25, 'non-neutral'],
 [26, 'anger'],
 [27, 'neutral'],
 [28, 'non-neutral']]

In [ ]:
sen

'hi pig'

In [ ]:
test_data[1:10]

(tensor([[  101,   183, 10237,  ...,     0,     0,     0],
         [  101, 21852,   102,  ...,     0,     0,     0],
         [  101,   102,     0,  ...,     0,     0,     0],
         ...,
         [  101, 11023, 12257,  ...,     0,     0,     0],
         [  101, 17860,   102,  ...,     0,     0,     0],
         [  101, 63001, 10718,  ...,     0,     0,     0]]),
 tensor([[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.]]),
 tensor([0, 1, 1, 2, 6, 2, 1, 0, 5]))

In [ ]:
nltk.download('stopwords')
import requests
from pandas import json_normalize
stops = set(stopwords.words('english'))
stemmer = nltk.stem.SnowballStemmer('english')
with open('./friends_test.json') as json_file:
    json_test = json.load(json_file)

def cleaning(str):
    replaceAll= str
    only_english = re.sub('[^a-zA-Z]', ' ', replaceAll)
    no_capitals = only_english.lower().split()
    no_stops = [word for word in no_capitals if not word in stops]
    stemmer_words = [stemmer.stem(word) for word in no_stops]
    #return ' '.join(stemmer_words)
    return ' '.join(stemmer_words)

test_data = []
for rows in json_test:
    for row in rows:
        test_data.append([cleaning(row['utterance']), row['emotion']])

en = pd.DataFrame(test_data, columns=['utterance','emotion'])     


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
en[10:30]

,utterance,emotion
10,well want,non-neutral
11,,non-neutral
12,know mayb mean mayb someth wrong,non-neutral
13,oh,non-neutral
14,man seen got,joy
15,hold,non-neutral
16,right man,joy
17,right congratul lucki bastard,joy
18,come lydia,neutral
19,push,joy


In [ ]:
model.cuda()

def test_sentences(sentences):
    model.eval()
    inputs, masks = convert_input_data(sentences)

    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
        
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    return logits

def convert_input_data(sentences):
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

def inttolabel(idx):
    return {0:'non-neutral',
             1:'neutral', 
             2:'joy',
             3:'sadness',
             4:'fear',
             5:'anger',
             6:'surprise',
             7:'disgust'}[idx]

en_data = [['Id', 'Predicted']]

for idx in range(len(en['utterance'])):
  sen = en['utterance'][idx]
  logit = test_sentences([sen])
  
  en_data.append([idx, inttolabel(np.argmax(logit)), en['emotion'][idx]])

dataframe = pd.DataFrame(en_data)
dataframe.to_csv("result.csv", header=False, index=False, mode='w+')

In [ ]:
#logit = test_sentences(['i love it'])
logit = test_sentences(['i am sad'])
print(inttolabel(np.argmax(logit)))

non-neutral
